# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [ ]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Réponses aux questions préléminaires

1) La règle de séparation :  
    On prend le premier objet sur lequel il n'y a pas encore eut de séparation.  
      
2) La méthode de calcul de la borne supérieur :  
    On met touts les objets initialement (Toutes les variables sont à 1). Après on met les objets ou non suivant les choix des noeuds déjà parcouru, les noeuds non parcourus restent à 1. 
    (exemple pour 2 variables :  
    noeud 0 : UB = [1,1]  
    noeud 1 : choix x[1] = 1 , UB = [1,1]  
    noeud 2 : choix x[1] = 0 , UB = [0,1]  
    ...)  
  
3) Les tests de sondabilité :  
    TA : Vérifie que la relaxation est faisable.  
    TO : Vérifier que la solution trouvée est meilleure que la meilleure solution connue.  
    TR : Vérifier que la solution trouvée est entière.  
      
4) La stratégie d'exploration :  
    PSES : priorité toujours à gauche


### Récupération des données

In [ ]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [ ]:
function TestsSondabilite_LP(bornsup, x, modele, BestProfit, Bestsol, affichage)
    price, weight, capacity, ratio, indicesratio = modele
    TA, TO, TR = false, false, false
    if(sum(weight .* x) > capacity)#Test de faisabilite (si le poids des objets choisis excède la capacité totale)
        TA=true
        if affichage
            println("TA")
        end 
    elseif(bornsup <= BestProfit) #Test d'optimalite (si la la solution maximale possible est inférieure à la meilleure déjà obtenue)
        TO=true
        if affichage
            println("TO")
        end
    elseif( sum(price .* x) == bornsup) #Test de resolution (si la solution courante est maximale pour la branche courante (égale à la borne supérieure))
        TR=true
        if affichage
            println("TR")
        end
        #si la solution trouvé >= BestProfit alors on met à jour la solution et le profit optimal
        if (bornsup >= BestProfit)
            Bestsol = x
            BestProfit = bornsup
            if affichage
                println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            end
        end
    else
        if affichage
            println("non sondable")
        end
    end
    TA, TO, TR, Bestsol, BestProfit
end

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [ ]:

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, modele, affichage)
    price, weight, capacity, ratio, indicesratio = modele
    n = length(price)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(indicesratio[i] in listobjs))
            obj=indicesratio[i]
        end
        i+=1
    end
    if affichage
        println("\nbranch on object ", obj, "\n")
    end
    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes,affichage)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            if affichage
                println("\nFINISHED")
            end
            stop=true
        end
    else
        #the root node was sondable
        if affichage
                println("\nFINISHED")
        end
        stop=true
    end
    return stop 
end

In [ ]:

# calculate the upper bound
# option = 1 : use of bound 1 capacity * max r_i with r_i= price_i/ weight_i
# option = 2 : use of bound 2 resolve of linear relaxation of the problem
function CalculBorneSup!(option, x, listobjs, listvals, modele)
    price, weight, capacity, ratio, indicesratio = modele
    bs = 0
    
    #On prend en compte les objets déjà choisi
    for i in listobjs
        bs = bs + x[i] * price[i]
    end
    
    if option==1
        # On prend le plus grand ratio des objets que l'on n'a pas encore parcourue
        if (length(listvals) != length(x))
            imax = indicesratio[length(listvals)+1]
            maxratio = ratio[imax]
            bs = bs + capacity * maxratio
        end
    else
        capaciteRestante = capacity
        
        for i in indicesratio 
            if(!(i in listobjs))
                if (weight[i] >= capaciteRestante)
                    bs = bs + capaciteRestante * ratio[i]
                    capaciteRestante = 0
                else
                    bs = bs + price[i]
                    capaciteRestante = capaciteRestante - weight[i]
                end
            end
        end
    end
    return bs
end

function Update_x(x, listvals, listobjs)
    x = zeros(length(x))
    for i in 1:length(listvals)
        x[listobjs[i]] = Float64.(listvals[i])
    end
    return x
end

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [ ]:

function SolveKnapInstance(filename, option; affichage = false)

    price, weight, capacity = readKnaptxtInstance(filename)
    ratio = price ./ weight
    indicesratio = sortperm(ratio, rev = true)
    
    modele = [price, weight, capacity, ratio, indicesratio]

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]
    x = zeros(length(price)) #initialisation of the solution of the current node

    current_node_number=0
    stop = false

    while(!stop)
        if affichage
            println("\nNode number ", current_node_number, ": \n---------------\n")
            println("obj", listobjs, "val", listvals, "\n")
        end

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)      
        
        x = Update_x(x, listvals, listobjs) # update x with the values on the current node
        
        bornsup = CalculBorneSup!(option, x, listobjs, listvals, modele) # calculate the upper bound
        if affichage
            println("\n x ", x )
            println("\n bornsup ", bornsup )
        end
        
        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_LP(bornsup, x, modele, BestProfit, Bestsol, affichage)

        is_node_sondable = TA || TO || TR

        # Pour la statégie d'exploration et la règle de séparation, nous avons gardé ce qui était initialement c'est à dire
        # exploration : PSES priorité d'exploration de l'arbre à gauche
        # séparation : On prend l'objet du plus grand ratio qui n'a pas déjà été étudié.
        if(!is_node_sondable)
            SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, modele, affichage)
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, affichage)
        end

        current_node_number = current_node_number + 1
    end
    if affichage
        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    end
    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


### Choix de structure de donnée

On donne pour les tests de sondabilité : bornsup, x, price, weight, capacity, BestProfit, BestSol
Pour pouvoir comparer la meilleur solution trouvée, à celle du noeud courant (x, price et BestProfit) afin de mettre à jour la solution.
Vérifier que la sol est correcte (x, weight et capacity) ou que le noeud courant ne permet pas d'atteindre la solution optimale (bornsup, BestProfit).
On a besoin de BestSol, pour la renvoyer si elle n'est pas mise à jour

Pour la règle de séparation on prend l'objet du plus grand ratio qui n'a pas déjà été étudié et la statégie d'exploration, nous avons gardé les mêmes données qu'avant

### Affichage du résultat final

In [ ]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt", 1, affichage = true)
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol, "\n Number of node : ", length(trNamenodes))
graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)

In [ ]:
function test(name)
    fichier = "instancesETU/KNAPnewformat/" * name
    println("Avec borne 1")
    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance(fichier, 1)
    println("\nNumber of node with bound 1 : ", length(trNamenodes))

        println("\n Avec borne 2")
    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance(fichier, 2)
    println("\nNumber of node  with bound 2 : ", length(trNamenodes), "\n.......\n")
    
end

In [ ]:
println("\n---------------------------------------\n")
println("50 données weakly_correlated\n")
test("weakly_correlated/knapPI_2_50_1000_1_-1396.opb.txt")
test("weakly_correlated/knapPI_2_50_10000_1_-13887.opb.txt")

# 100 données weakly_correlated (l'execution du test est beaucoup trop longue)
# test("weakly_correlated/knapPI_2_100_10000_5_-34346.opb.txt")

# 50 données almost_strongly_correlated(l'execution du test est beaucoup trop longue)
#test("almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt")


println("\n---------------------------------------\n")
println("20 données circle\n")
test("circle/knapPI_16_20_1000_1_-2291.opb.txt")

println("50 données circle\n")
test("circle/knapPI_16_50_1000_1_-3408.opb.txt")


println("\n---------------------------------------\n")
println("50 données inverse_strongly_correlated\n")
test("inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt")

println("100 données inverse_strongly_correlated\n")
test("inverse_strongly_correlated/knapPI_4_100_1000_1_-997.opb.txt")

# 200 données inverse_strongly_correlated (l'execution du test est beaucoup trop longue) \n"
# test("inverse_strongly_correlated/knapPI_4_200_1000_3_-3152.opb.txt")

println("\n---------------------------------------\n")
println("20 données multiple_strongly_correlated\n")
test("multiple_strongly_correlated/knapPI_14_20_1000_1_-1794.opb.txt")

# 50 données multiple_strongly_correlated (l'execution du test est beaucoup trop longue)
# test("multiple_strongly_correlated/knapPI_14_50_1000_5_-3237.opb.txt")

println("\n---------------------------------------\n")
println("20 données profit_ceiling\n")
test("profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt")

# 50 données profit_ceiling(l'execution du test est beaucoup trop longue)
# test("profit_ceiling/knapPI_15_50_1000_5_-1047.opb.txt")

println("\n---------------------------------------\n")
println("50 données similar_weights\n")
test("similar_weights/knapPI_9_50_1000_1_-995.opb.txt")

println("100 données similar_weights\n")
test("similar_weights/knapPI_9_100_1000_1_-995.opb.txt")

println("200 données similar_weights\n")
test("similar_weights/knapPI_9_200_1000_1_-995.opb.txt")

# 500 données similar_weights(l'execution du test est beaucoup trop longue)
# test("similar_weights/knapPI_9_500_1000_1_-3974.opb.txt")

### Interprétation des tests

| cas de test | Nb données| Borne 1 | Borne 2 |
|:-: |:-: |:-: | :-: |
| weakly_correlated | 50 | 180889| 63797 |
| weakly_correlated | 50 | 20307| 6067 |
| circle | 20 | 2501 | 1941 |
| circle | 50 | 2501 | 1941 |
| inverse_strongly_correlated | 50 | 101 | 101 |
| inverse_strongly_correlated | 100 | 201 | 201 |
| multiple_strongly_correlated | 20 | 2433 | 1263 |
| profit_ceiling | 20 | 2309 | 2007 |
| similar_weights | 50 | 101 | 101 |
| similar_weights | 100 | 201 | 201 |
| similar_weights | 200 | 26011 | 25815 |


En général l'algorithme avec la borne 1 se résoud en plus de noeud qu'avec la borne 2. Ce qui est cohérent car la borne 2 est plus restreignante.
On remarque que pour les cas inverse_strongly_correlated et similar_weights, pour des nombre de donné de 50 et 100, il y a un nombre de noeud identique pour les 2 bornes. Cela est cohérent car avec des ratios qui sont similaires pour toutes les données, le calcul de borne est très proche.